https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/export/?disjunctive.cou_name_en&sort=name&location=7,51.62484,6.58081&basemap=jawg.streets


In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
pop = pd.read_csv("data/processed/nl3.csv", sep=',',index_col=[0])
dash = pd.read_csv("app/df_coo_pararius.csv")
zio = pd.read_csv("data/processed/zipcodeZ.csv", sep=',',index_col=[0])

In [3]:
searchfor = dash['Plaats'].unique()
pop['Alternate Names'] = pop['Alternate Names'].fillna('None')
for i in searchfor:
    pop.loc[pop['Alternate Names'].str.contains(i),'city'] = i
pop['city'] = pop['city'].fillna(pop['city'])

<ipython-input-3-8065dc9aa161>:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  pop.loc[pop['Alternate Names'].str.contains(i),'city'] = i


In [4]:
ast  =  list(set(searchfor) - set(pop['city']))

In [5]:
test = dash[dash['Plaats'].isin(ast)]
aaa = pd.merge(test[['Plaats','price']],zio[['Plaats','Provincie','longitude','latitude']], left_on='Plaats', right_on='Plaats',how='left')
ccc = aaa.groupby(['Plaats','Provincie'])[['longitude','latitude']].mean().reset_index()

In [6]:
def haversine(lat1, lon1, lat2, lon2):
    earth_radius=6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    return earth_radius * 2 * np.arcsin(np.sqrt(a))

ccc['latitude1'] = 51.907183
ccc['longitude1'] = 4.472815
ccc['distance'] = haversine(ccc['latitude'], ccc['longitude'], ccc['latitude1'], ccc['longitude1'])
ccc = ccc.rename(columns={'Plaats':'city','Provincie':'Province'})

In [7]:
finaleira = pd.concat([pop,ccc],0).reset_index(drop=True)

<ipython-input-7-cc23d04fda52>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  finaleira = pd.concat([pop,ccc],0).reset_index(drop=True)


In [8]:
import numpy as np
def haversine(lat1, lon1, lat2, lon2):
    earth_radius=6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    return earth_radius * 2 * np.arcsin(np.sqrt(a))

finaleira['latitude1'] = 51.907183
finaleira['longitude1'] = 4.472815
finaleira['distance'] = haversine(finaleira['latitude'], finaleira['longitude'], finaleira['latitude1'], finaleira['longitude1'])
finaleira = finaleira.sort_values('distance')

In [9]:
finaleira['cost'] = finaleira['cost'].interpolate().fillna(finaleira['cost'].mean())

In [10]:
finaleira.to_csv("data/processed/nl3.csv")

In [11]:
average_rent = dash.groupby(['Plaats','Provincie'])[['price']].mean().reset_index().rename(columns={'price':'avg_rent'})
fim = pd.merge(finaleira,average_rent,left_on=['city','Province'],right_on=['Plaats','Provincie'],how='left')
fim['avg_costs'] = fim['avg_rent'] + fim['cost']

In [12]:
fim['latitude' ] = round(fim['latitude'].astype(float),5)
fim['longitude'] = round(fim['longitude'].astype(float),5)
fim['distance'] = round(fim['distance'],2)
fim['cost'] = round(fim['cost'],2)
fim['avg_rent'] = round(fim['avg_rent'],2)
fim['avg_costs'] = round(fim['avg_costs'],2)

In [14]:
final = fim[[
    'Alternate Names',
    'city',
    'Province',
    'Population',
    'distance',
    'cost',
    'avg_rent',
    'avg_costs',
    'latitude',
    'longitude',
]].fillna('None')
final

,Alternate Names,city,Province,Population,distance,cost,avg_rent,avg_costs,latitude,longitude
0,"RTM,Ratehrdam,Roterdam,Roterdama,Roterdamas,Ro...",Rotterdam,Zuid-Holland,598199.0,1.76,857.05,1509.56,2366.61,51.92250,4.47917
1,"Schidamas,Schiedam,Skhidam,Skiedam,Sxidam,askh...",Schiedam,Zuid-Holland,75438.0,5.91,857.05,1239.46,2096.51,51.91917,4.38889
2,"Barendrekht,Barendrext,ba lun de lei he te,bar...",Barendrecht,Zuid-Holland,38672.0,7.04,847.63,1747.5,2595.13,51.85667,4.53472
3,"Capelle,Capelle a/d IJssel,Capelle a/d Yssel,C...",Capelle aan den IJssel,Zuid-Holland,65255.0,7.60,842.92,1220.29,2063.21,51.92917,4.57778
4,Hoogvliet Rotterdam,Hoogvliet Rotterdam,Zuid-Holland,34950.0,9.01,835.85,1226.67,2062.52,51.86333,4.36250
...,...,...,...,...,...,...,...,...,...,...
454,None,Farmsum,Groningen,2185.0,228.40,877.61,0.0,877.61,53.32167,6.92639
455,None,Roodeschool,Groningen,None,228.48,877.61,1050.0,1927.61,53.41983,6.76638
456,"Delfzejl,Delfzijl,Delfzyl,Делфзейл",Delfzijl,Groningen,28649.0,228.62,877.61,680.6,1558.21,53.33000,6.91806
457,Spijk Gn,Spijk,Groningen,1220.0,229.40,877.61,1500.0,2377.61,53.39000,6.83750


In [15]:
searchfor = ["'s-Hertogenbosch",
 'Alkmaar',
 'Almere',
 'Amersfoort',
 'Amstelveen',
 'Amsterdam',
 'Apeldoorn',
 'Arnhem',
 'Breda',
 'Bussum',
 'Delft',
 'Dordrecht',
 'Eindhoven',
 'Gouda',
 'Haarlem',
 'Hilversum',
 'Hoofddorp',
 'Huizen',
 'Leiden',
 'Oegstgeest',
 'Rijswijk',
 'Rotterdam',
 'Schiedam',
 'Den Haag',
 'Utrecht',
 'Voorburg',
 'Zaandam',
 'Zeist',
 'Zoetermeer']

In [18]:
final[final['city'].isin(searchfor)].reset_index(drop=True).to_csv('app/costs.csv')

,Alternate Names,city,Province,Population,distance,cost,avg_rent,avg_costs,latitude,longitude
0,"RTM,Ratehrdam,Roterdam,Roterdama,Roterdamas,Ro...",Rotterdam,Zuid-Holland,598199.0,1.76,857.05,1509.56,2366.61,51.92250,4.47917
1,"Schidamas,Schiedam,Skhidam,Skiedam,Sxidam,askh...",Schiedam,Zuid-Holland,75438.0,5.91,857.05,1239.46,2096.51,51.91917,4.38889
2,"Delft,Ntelpht,dai er fu te,derufuto shi,dlpt,Ν...",Delft,Zuid-Holland,95060.0,13.67,788.75,1272.57,2061.32,52.00667,4.35556
3,"Zeutermaer,Zoetermeer,zwtrmyyr,זוטרמייר",Zoetermeer,Zuid-Holland,115845.0,16.77,839.29,989.12,1828.41,52.05750,4.49306
4,"Doardt,Dordracum,Dordrech,Dordrecht,Dordrechta...",Dordrecht,Zuid-Holland,119260.0,17.52,835.52,1134.5,1970.02,51.81000,4.67361
5,None,Rijswijk,Zuid-Holland,47299.0,17.57,847.98,1654.2,2502.18,52.03634,4.32501
6,"Voorburg,fwrbwrkh,Воорбург,فوربورخ",Voorburg,Zuid-Holland,38000.0,20.12,941.47,1329.83,2271.3,52.07417,4.35972
7,"Chaounta,Gauda,Gaudanum,Gouda,Guda,Quda,ga'uda...",Gouda,Zuid-Holland,71952.0,20.21,771.80,1134.0,1905.8,52.01667,4.70833
8,None,Den Haag,Zuid-Holland,None,22.80,811.32,1843.61,2654.92,52.08123,4.29669
9,"Lajden,Leida,Leiden,Leidenas,Leidene,Leie,Leie...",Leiden,Zuid-Holland,119713.0,27.96,868.50,1462.82,2331.32,52.15833,4.49306


In [19]:
pd.read_csv('app/costs.csv', index_col=[0])

,Alternate Names,city,Province,Population,distance,cost,avg_rent,avg_costs,latitude,longitude
0,"RTM,Ratehrdam,Roterdam,Roterdama,Roterdamas,Ro...",Rotterdam,Zuid-Holland,598199.0,1.76,857.05,1509.56,2366.61,51.92250,4.47917
1,"Schidamas,Schiedam,Skhidam,Skiedam,Sxidam,askh...",Schiedam,Zuid-Holland,75438.0,5.91,857.05,1239.46,2096.51,51.91917,4.38889
2,"Delft,Ntelpht,dai er fu te,derufuto shi,dlpt,Ν...",Delft,Zuid-Holland,95060.0,13.67,788.75,1272.57,2061.32,52.00667,4.35556
3,"Zeutermaer,Zoetermeer,zwtrmyyr,זוטרמייר",Zoetermeer,Zuid-Holland,115845.0,16.77,839.29,989.12,1828.41,52.05750,4.49306
4,"Doardt,Dordracum,Dordrech,Dordrecht,Dordrechta...",Dordrecht,Zuid-Holland,119260.0,17.52,835.52,1134.5,1970.02,51.81000,4.67361
5,None,Rijswijk,Zuid-Holland,47299.0,17.57,847.98,1654.2,2502.18,52.03634,4.32501
6,"Voorburg,fwrbwrkh,Воорбург,فوربورخ",Voorburg,Zuid-Holland,38000.0,20.12,941.47,1329.83,2271.3,52.07417,4.35972
7,"Chaounta,Gauda,Gaudanum,Gouda,Guda,Quda,ga'uda...",Gouda,Zuid-Holland,71952.0,20.21,771.80,1134.0,1905.8,52.01667,4.70833
8,None,Den Haag,Zuid-Holland,None,22.80,811.32,1843.61,2654.92,52.08123,4.29669
9,"Lajden,Leida,Leiden,Leidenas,Leidene,Leie,Leie...",Leiden,Zuid-Holland,119713.0,27.96,868.50,1462.82,2331.32,52.15833,4.49306
